### **6.9 - [Practica] Templating your tasks**

https://airflow.apache.org/docs/apache-airflow/stable/templates-ref.html

En el vídeo anterior hemos visto los conceptos de variables, templates y macros.  En este vídeo vamos a mezclar todas estas cosas para que entiendas cómo utilizarlas y las posibilidades que te ofrece Airflow.  Primero, desde tu editor de código, comprueba que estás en la carpeta airflow-materials/airflow-section-6 con el archivo template_dag.py abierto.  Ok, no te preocupes, sé que este DAG parece difícil de entender, pero voy a explicar exactamente lo que hace paso a paso. 

<center><img src="https://i.postimg.cc/pdPt7MN6/a1004.png"></center>
<center><img src="https://i.postimg.cc/44YnwVQh/a1005.png"></center>

La ruta sys insertada aquí, es sólo una forma rápida de poder importar las funciones python del módulo process_logs en la carpeta "scripts".  

<center><img src="https://i.postimg.cc/JnG76CZW/a1006.png"></center>

Entonces, por ahora, nos saltamos el código en comentario, Volveremos a él en el momento adecuado.  Nada nuevo de los “default arguments”, deberías estar familiarizado con ellos.  

<center><img src="https://i.postimg.cc/13Qt9Thp/a1007.png"></center>

___

#### **`Templates`**

Luego tenemos la instancia del DAG con las tareas que se van a ejecutar.  **`Si recuerdas, dije antes que sólo algunos parámetros están templados en sus operadores`**.  Puede que te preguntes "Vale, eso está muy bien, pero ¿cómo sé qué operador tiene parámetros templados (templated parameters)?".  Bueno, para encontrar la respuesta, tienes que echar un vistazo a la documentación de Airflow.  Así que digamos que queremos utilizar el BashOperator para la primera tarea, 

<center><img src="https://i.postimg.cc/ry7sHsfZ/a1008.png"></center>

¿cómo puedo saber qué parámetros se pueden utilizar con templates.  Abra su navegador web, y escriba "https://airflow.apache.org/docs/apache-airflow/stable/_api/airflow/operators/index.
html". 

<center><img src="https://i.postimg.cc/ZqY5Ndqn/a1009.png"></center>

Desde ahí tenemos la lista completa de operadores oficiales que puedes añadir a tu DAG.  Está el BashOperator, el DockerOperator, el EmailOperator y así sucesivamente. En nuestro caso nos vamos a centrar en el BashOperator así que vamos a hacer clic sobre él. 

<center><img src="https://i.postimg.cc/zB6z42FH/a1010.png"></center>

Desde esta página, obtenemos diferente información sobre el operador como por ejemplo cómo puede instanciarse, la descripción de sus parámetros y demás.  Para nosotros, la parte interesante es la sección "template_fields".  Esto nos da la lista de los templated parameters para el BashOperator.  Esto significa que podemos ejecutar un comando bash con valores añadidos en tiempo de ejecución utilizando las llaves como marcadores de posición (using the curly bracket place holders).  

<center><img src="https://i.postimg.cc/q7mMxvtL/a1011.png"></center>

Bien, déjame mostrarte un ejemplo concreto.  De vuelta a tu editor de código, la tarea "t0", implementa el BashOperator para ejecutar un comando muy simple para mostrar desde el output la execution date del DAG. 

<center><img src="https://i.postimg.cc/Hk9Lj2dF/a1012.png"></center>

Aquí, las llaves indican al Jinja template engine que deberá reemplazar el valor "ds" por la execution date del DAG.  Observa que “ds” es un valor predefinido.  Así que vamos a comprobar si funciona.  Abre tu terminal y comprueba que estás en la carpeta airflow-materials/airflow-section-6. Desde ahí, inicia Airflow escribiendo "docker-compose - f docker-compose-CeleryExecutor.yml up -d". Enter. Ok, ahora que los contenedores docker se están ejecutando, vaya a su navegador web, y acceda a la interfaz de usuario de Airflow escribiendo localhost:8080.  Activa el toggle del DAG “template_dag” y espera a que termine el DAGRun.  Ok, haz click en el DAG, "Graph View", "t0" y "View Log". Si echas un vistazo a la parte inferior de los registros, podrás ver la salida del comando bash junto con la execution date del DAG.  

<center><img src="https://i.postimg.cc/rFSVYJ0z/a1013.png"></center>

___

#### **`Creación de una variable`**

Por supuesto, no tendrás la misma fecha ya que depende de cuando ejecutes el DAG en tu máquina, pero así es como puedes obtener un valor dinámico de tu DAG usando templates.  Recuerda que en nuestro código sólo hemos puesto la variable "ds". Esta variable ha sido reemplazada por el "template engine" en tiempo de ejecución con la fecha de ejecución actual del DAG.  Bien, ahora avancemos un paso.  De vuelta a la interfaz de usuario, vamos a crear una variable. Para ello, haga clic en "Admin", "Variables" y "Create". Aquí, necesitamos especificar una key, digamos CASSANDRA_LOGIN.  Para el valor escribimos "my_login" y pulsamos en "Guardar". 

<center><img src="https://i.postimg.cc/h4QPVgKL/a1014.png"></center>

Como puedes ver, tenemos una nueva variable llamada "CASSANDRA_LOGIN" con el valor "my_login" que esta encriptada por defecto. 

<center><img src="https://i.postimg.cc/pXNVZ7dg/a1015.png"></center>

Mira cómo podemos recuperarla del DAG. Volviendo al editor de código, sólo tenemos que sustituir "ds" por "var.value." y el nombre de la variable que queramos.  En nuestro caso es CASSANDRA_LOGIN.  Eso es todo.  Cada vez que quieras acceder a tus variables usando el template engine desde la base de datos de Airflow, tienes que empezar con el prefijo "var.value" y luego la key de la variable. Guarde el archivo. 

<center><img src="https://i.postimg.cc/13smxqwB/a1016.png"></center>

A continuación, desde la interfaz de usuario, haga clic en "DAGs", "template_dag", actualice el DAG y active otro DAGRun haciendo clic aquí.  Espera a que el DAGRun termine. Ok. Haga clic en la tarea, "View Log", y en la parte inferior de los logs, se puede ver que esta vez, tenemos el valor de la variable CASSANDRA_LOGIN que es "my _login" en lugar de la execution date del DAG.  

<center><img src="https://i.postimg.cc/g0Bm6t5t/a1017.png"></center>

___

#### **`Macros`**

Bien, ya hemos visto cómo tratar con variables predefinidas y variables que nosotros creamos, veamos como llamar a una macro en un parámetro templado para modificar un valor dinámicamente. En tu editor de código, sustituye el valor entre las llaves por la macro definida en la instrucción deshabilitada TEMPLATED_LOG_DIR.  Entonces copia este valor hasta el paréntesis, y pégalo.  No olvides reemplazar las comillas dobles por comillas simples.  

<center><img src="https://i.postimg.cc/Dzz703Cb/a1018.png"></center>
<center><img src="https://i.postimg.cc/zG7JqGkr/a1019.png"></center>

Entonces, ¿qué hace esta macro? Al escribir "macros." 

<center><img src="https://i.postimg.cc/sXQyP9Cj/a1020.png"></center>

se obtiene un acceso a las macros predefinidas de Airflow. Si desea consultar la lista exhaustiva, no dude en consultar el siguiente enlace. La macro "ds_format" toma un string de entrada que es otra variable predefinida llamada "ts_nodash" que corresponde a la execution date del DAG en formato iso, luego, macro "ds_format" emite (outputs) otro string como se especifica en el formato de salida. Así, en nuestro caso, la execution date dada sin guiones con el formato definido aquí, 

<center><img src="https://i.postimg.cc/gj3WQz01/a1021.png"></center>

se transformará para que coincida con el formato de salida dado aquí.  

<center><img src="https://i.postimg.cc/RCt4NRTG/a1022.png"></center>

Básicamente, añadimos guiones entre año, mes, día y así sucesivamente.  Vamos a añadir "ts_nodash" justo antes de la llamada de la macro para que se vea el valor original.  No olvides poner también las llaves.  Ok, guarda el archivo y vuelve a la interfaz de usuario. 

<center><img src="https://i.postimg.cc/GhJRjmVy/a1023.png"></center>

Desde allí, haz clic en "actualizar" y luego en "trigger DAG". Espera a que el DAG termine.  Ahora haga clic en la tarea y "View log".  Como puedes ver, el valor original dado por "ts_nodash" es la siguiente fecha y una vez que "ts_nodash" es transformado por la macro "ds_format", obtenemos la siguiente fecha, esta vez con guiones.  

<center><img src="https://i.postimg.cc/13vZLY9R/a1024.png"></center>

___

#### **`Archivos templados`**

Así que aquí, además de usar variables predefinidas, también las modificamos usando macros. Bastante potente, ¿verdad?  Pero déjame enseñarte una cosa más.  ¿Sabías que tus archivos también pueden ser templados?  De hecho, si echas un vistazo a la documentación de Airflow sobre el BashOperator, puedes ver justo debajo de los parámetros templados, la sección template_ext con dos valores ".sh" y ".bash".  

<center><img src="https://i.postimg.cc/WbPjrkWV/a1025.png"></center>

Esto significa que, si llamas a un archivo script desde el parámetro de comando bash, también puedes incluir marcadores de posición de template en ese archivo script.  Déjeme mostrarle un ejemplo. Descomente la tarea t1 y guarde el archivo. 

<center><img src="https://i.postimg.cc/FHr4CQXn/a1026.png"></center>

t1 ejecutará el script “generate_new_logs.sh” para generar logs falsos en el archivo llamado “log.csv” que puedes encontrar desde el parámetro params aquí. Este parámetro es una referencia al diccionario de parámetros (params dictionary) definidos por el usuario en la tarea de llamada.  Está disponible para todos los operadores y puedes acceder a él desde una tarea utilizando templates o el contexto de la tarea (context of the task).  Por ejemplo, aquí hemos especificado un par clave-valor "filename", "log.csv". Al hacerlo, este valor pasa a estar disponible en el contexto del operador.  

Si abre el archivo “generate_new_logs.sh” en la carpeta script, encontrará el marcador de posición “params.filename” justo ahí, que será reemplazado por el valor "log .csv" en tiempo de ejecución. 

<center><img src="https://i.postimg.cc/XvX03cJf/a1027.png"></center>

También, si echamos un vistazo a la línea DEST_PATH, puedes notar que los valores son templados directamente en el archivo.  Así que aquí, tenemos una variable llamada "source_path" necesaria para especificar la ruta donde se almacenará el archivo log generado.  

<center><img src="https://i.postimg.cc/9fSHfRRv/a1028.png"></center>

Como ejercicio rápido, quiero que cree esta variable desde la interfaz de usuario de Airflow, con la key “source_path” y el valor “/usr/local/airflow/dags”.  Ponga el vídeo en pausa y vuelva cuando esté listo. Desde la interfaz de usuario, haga clic en "admin", "variables", "create" y aquí sólo tienes que escribir "source_path" para la key y "/usr/local/airflow/dags" para el valor.  "Guardar".  

<center><img src="https://i.postimg.cc/bJtcSy1d/a1029.png"></center>

Ahora si hacemos click en "DAGs", luego en “template_dag”. Refresca el DAG y vamos a dispararlo.  Ahora espera hasta que el DAGRun termine.  Ok, ahora si volvemos al editor de código, tenemos una nueva carpeta llamada "data" con un archivo log generado.  Este archivo ha sido generado por el script “generate_new_logs.sh” que hemos visto anteriormente en la ubicación templada (templated location). Ábrelo. Hay 4 columnas "index", "message", "timestamp" y "ds_airflow". 

<center><img src="https://i.postimg.cc/PJWgKS3j/a1030.png"></center>

El contenido de este archivo no es realmente importante excepto por la última columna donde el valor es en realidad la execution date del DAG que también se obtiene del archivo de script "generate_new_logs.sh".  

<center><img src="https://i.postimg.cc/2yNp4sxG/a1031.png"></center>

Permítanme mostrarles una característica muy interesante de Airflow.  Si selecciona la última tarea,

<center><img src="https://i.postimg.cc/66Cx9Q9m/a1032.png"></center>
<center><img src="https://i.postimg.cc/HsHDjCns/a1033.png"></center>

y hace clic en "Rendered", obtendrá el template procesado por el template engine con los valores finales de sus placeholders (marcadores de posición). 

<center><img src="https://i.postimg.cc/15HxqYV0/a1034.png"></center>

Por ejemplo, en la variable DEST_PATH tenemos la ruta "/usr/local/airflow/dags" obtenida de la variable source _path. El nombre del archivo es la “execution date” del DAG como se muestra aquí. 

<center><img src="https://i.postimg.cc/rwnvh0JK/a1035.png"></center>

El archivo de destino llamado “log.csv” está bien definido a partir de los parámetros “param” especificados en el BashOperator y así sucesivamente.  

<center><img src="https://i.postimg.cc/g2kCqq3C/a1036.png"></center>

Esta es una gran manera de comprobar lo que sus archivos o comandos templated producirán y le evitará algunos errores. Ok, antes de pasar al siguiente video, me gustaría mostrarte una última cosa.  De vuelta a nuestro DAG, descomenta la instrucción TEMPLATE_LOG_DIR, así como la tarea t2 que sólo comprueba si el archivo log ha sido generado.  Y finalmente, descomenta la tarea t3 que hace algunas modificaciones en los logs, así como en las dependencias.  

<center><img src="https://i.postimg.cc/N0KZZsJb/a1037.png"></center>
<center><img src="https://i.postimg.cc/PxZ9r6kw/a1038.png"></center>

___

#### **`provide_context`**
#### **`templates_dict`**

Ok, hay dos cosas a tener en cuenta aquí. PythonOperator proporciona dos parámetros importantes: "provide_context" y "templates_dict". "**`provide_context`**" cuando se establece en "**`True`**", permite a Airflow pasar un conjunto de argumentos de palabras clave (keyword arguments) que se pueden utilizar en su función.  Básicamente, es una manera de obtener información acerca de su DAG, su tarea y así sucesivamente, exactamente como lo hacemos con los templates utilizando las variables predefinidas, "ds", "ti" y así sucesivamente.  **`Tenga en cuenta que este parámetro se establece por defecto en True, por lo que no necesitará especificarlo más`**.  El parámetro "**`templates_dict`**" es un diccionario donde los valores serán templados por el motor Jinja y estarán disponibles en el contexto proporcionado.  Ok, esto puede sonar difícil, así que déjame mostrarte un ejemplo.  Digamos que queremos pasar la ruta TEMPLATED_LOG_DIR que es la ruta del archivo log generado. Para ello, primero tenemos que establecer en “True” el parámetro "provide_context" y luego crear el par clave-valor con la key "log_dir" y el valor correspondiente a la ruta templada.

<center><img src="https://i.postimg.cc/rp77kB7T/a1039.png"></center>

Entonces, si abres el archivo "process_logs.py", 

<center><img src="https://i.postimg.cc/8cS3Fh1n/a1040.png"></center>

las dos líneas de aquí es donde ocurre la magia.  

<center><img src="https://i.postimg.cc/4yQ0pmy4/a1041.png"></center>

Primero, obtenemos el valor “log_dir” accediendo al contexto proporcionado desde el parámetro de la función como se muestra aquí. Como el contexto tiene mucha información sobre la tarea, especificamos que sólo queremos acceder al “templates_dict”.  Luego, para obtener el nombre del archivo, accedemos de nuevo al contexto pero esta vez desde params con la key "filename". Y ya está.  Ya tenemos toda la información que necesitamos. No te preocupes por el contenido de esta función, sólo aplica algunas modificaciones a los logs como renombrar las columnas y formatear el timestamp usando el módulo Pandas.  Así que vamos a activar el DAG para ver lo que obtenemos. Haz click en "DAGs", "template_ dag", "refresh" y "trigger_dag".  Empieza a refrescar la página hasta que termine el DAGRun.  Ok.  Ahora si volvemos al editor de código, podemos ver que tenemos un nuevo archivo llamado “processed_log.csv” al lado del archivo “log.csv” como indica la variable templada TEMPLATED _LOG_DIR. 

<center><img src="https://i.postimg.cc/9XdK6Nb3/a1042.png"></center>

Bien, una última cosa y prometo que hemos terminado.  El parámetro "params" no acepta valor templado.  Así que no intentes poner un placeholder (marcador de posición) con llaves, no funcionará.  Comentemos las tareas t3, t2, así como TEMPLATED_LOG_DIR de nuevo para evitar tener errores cuando reiniciemos Airflow ya que la variable source_path ya no existirá.  Guarde el archivo. Ahora, vaya a su terminal y detenga los contenedores docker escribiendo "docker-compose - f docker-compose-CeleryExecutor.yml down".  Así que, espero que hayas disfrutado de este video, fue uno bastante difícil con una gran cantidad de información.  Tómate tu tiempo para hacer tus propios experimentos y nos vemos en el próximo vídeo.  